<a href="https://colab.research.google.com/github/ganeshbulagondla/content/blob/main/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
!pip install pyPDF2
!pip install sentence_transformers
!pip install langchain
!pip install langchain-community
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        str: Extracted text from the PDF.
    """
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text
def chunk_text(text, chunk_size=500, chunk_overlap=50):
    """
    Splits text into chunks of a specified size with overlap.

    Args:
        text (str): The input text.
        chunk_size (int): The size of each chunk.
        chunk_overlap (int): The number of overlapping characters between chunks.

    Returns:
        List[str]: List of text chunks.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_text(text)
def generate_embeddings(chunks, embedding_model):
    """
    Generates embeddings for text chunks using a pre-trained model.

    Args:
        chunks (List[str]): List of text chunks.
        embedding_model (SentenceTransformer): Pre-trained embedding model.

    Returns:
        List[np.ndarray]: List of vector embeddings.
    """
    return [embedding_model.encode(chunk) for chunk in chunks]
def store_embeddings_in_faiss(chunks, embeddings):
    """
    Stores text chunks and their embeddings in a FAISS vector database.

    Args:
        chunks (List[str]): List of text chunks.
        embeddings (List[np.ndarray]): Corresponding embeddings for the chunks.

    Returns:
        FAISS: A FAISS vector database instance.
    """
    faiss_db = FAISS.from_texts(texts=chunks, embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
    return faiss_db
def data_ingestion_pipeline(pdf_dir, embedding_model_name="sentence-transformers/all-MiniLM-L6-v2"):
    """
    Executes the data ingestion pipeline: text extraction, chunking, embedding generation, and storage.

    Args:
        pdf_dir (str): Path to the directory containing PDF files.
        embedding_model_name (str): Name of the pre-trained embedding model.

    Returns:
        FAISS: A FAISS vector database instance with stored embeddings.
    """
    embedding_model = SentenceTransformer(embedding_model_name)
    all_chunks = []

    for pdf_file in os.listdir(pdf_dir):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_dir, pdf_file)
            print(f"Processing: {pdf_path}")

            text = extract_text_from_pdf(pdf_path)
            chunks = chunk_text(text)
            all_chunks.extend(chunks)
    embeddings = generate_embeddings(all_chunks, embedding_model)
    faiss_db = store_embeddings_in_faiss(all_chunks, embeddings)

    return faiss_db
if _name_ == "_main_":
    pdf_directory = "path/to/your/pdf/files"
    vector_db = data_ingestion_pipeline(pdf_directory)
    print("Data ingestion completed. Vector database is ready.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
